# Poincloud Processing

The Camera Transformation Matrix: The transformation that places the camera in the correct position and orientation in world space (this is the transformation that you would apply to a 3D model of the camera if you wanted to represent it in the scene).

In [1]:
import open3d as o3d
import numpy as np
import pandas as pd
import copy
import time
import os
import sys
import glob
import yaml
import pyvista as pv
import pymeshfix as mf

from dictionary_manager import get_serial_dictionary

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def custom_draw_geometry(pcd):
    #define the screen resolution
    screen_res = 1920, 1080
    window_size = 640, 480
    # The following code achieves the same effect as:
    # o3d.visualization.draw_geometries([pcd])
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=window_size[0], height=window_size[1], left=(screen_res[0]//2)-(window_size[0]//2), top=(screen_res[1]//2)-(window_size[1]//2))
    if isinstance(pcd, list):
        for point_cloud in pcd: 
            vis.add_geometry(point_cloud)
    else:
        vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()

In [4]:
def panel_draw(mesh):
    pl = pv.Plotter()
    pl.add_mesh(mesh, color='lightgrey')
    pl.background_color = 'white'
    pl.camera_position = 'xz'
    pl.show(window_size=[640, 480], jupyter_backend = 'panel')

In [5]:
def import_pointcloud(sequence_id, path):
    
    meshes = glob.glob(path + '/*.ply')
    
    file_name = path + '/' + 'transformation_matrix.yaml'
    with open(file_name, 'r') as file:
        transformation_matrix = yaml.safe_load(file)
        
    serial_mesh_path = get_serial_dictionary(meshes, transformation_matrix)
    
    mesh = {}
    for serial, paths in serial_mesh_path.items():
        mesh[serial] = o3d.io.read_point_cloud(paths)
        
    transformation = {}
    for serial, matrix in transformation_matrix.items():
        transformation[serial] = np.asarray(matrix)
        
    mesh_transform = {}
    point_clouds = []
    for serial, meshes in mesh.items():
        mesh_transform[serial] = copy.deepcopy(meshes).transform(transformation[serial])
        point_clouds.append(mesh_transform[serial])
        
    import_pointcloud.voxel_size = 0.002
    pcds_down = {}
    for serial, pcds in mesh_transform.items():
        pcds_down[serial] = pcds.voxel_down_sample(voxel_size=import_pointcloud.voxel_size)
    
    camera_location = {}
    for serial, translation in transformation.items():
        camera_location[serial] = translation[0:3,3]
        
    pcds_normals = {}
    for serial, pcds in pcds_down.items():
        pcds_normals[serial] = get_pc_normals(pcds, camera_location[serial])
    
    sequence_pcds = []
    for serial in sequence_id:
        sequence_pcds.append(pcds_normals[serial])

    return sequence_pcds

In [6]:
def pc_down_sample(pcds, voxel_size):
    
    pcds_down = []
    if isinstance(pcds, list):
        for point_cloud in pcds:
            pcd_temp = point_cloud.voxel_down_sample(voxel_size=voxel_size)
            pcds_down.append(pcd_temp)
        
    else:
        pcds_down = pcds.voxel_down_sample(voxel_size=voxel_size)       
        
    return pcds_down

In [7]:
def get_pc_normals(pcds_down, camera_loc):
    if isinstance(pcds_down, list):
        for point_cloud in pcds_down:
            point_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
            point_cloudpoint_cloud.orient_normals_towards_camera_location(camera_location=camera_loc)
    else:
        pcds_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        pcds_down.orient_normals_towards_camera_location(camera_location=camera_loc)
    
    
    return pcds_down

In [10]:
def pairwise_registration(source, target):
    coarse_threshold = import_pointcloud.voxel_size
    fine_threshold = coarse_threshold * 0.25
    current_transformation = np.identity(4)
    icp_coarse = o3d.pipelines.registration.registration_icp( source, target, coarse_threshold, current_transformation, o3d.pipelines.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.pipelines.registration.registration_icp(source, target, fine_threshold, icp_coarse.transformation, o3d.pipelines.registration.TransformationEstimationPointToPlane())

    return icp_fine.transformation

In [11]:
def combine_pcds(source, target):
    pcd_combined = o3d.geometry.PointCloud()
    pcd_combined = source + target
    
    return pcd_combined

In [12]:
def full_registration(pcds):
    new_pc = pcds[0]
    for n in range(len(pcds)-1):
        transformation = pairwise_registration(new_pc, pcds[n+1])
        target_temp = copy.deepcopy(pcds[n+1])
        new_pc.transform(transformation)
        combined = combine_pcds(new_pc, target_temp)
        new_pc = combined
        
    return new_pc

In [11]:
def reconstruction(pc, patient, path): # Depth relates to the mesh quality, the bigger = more details
    poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pc, depth=6, width=0, scale=1.1, linear_fit=False)[0] 
    bbox = pc.get_axis_aligned_bounding_box()
    p_mesh_crop = poisson_mesh.crop(bbox)
    
    filename = patient + '_mesh_poisson.ply'
    o3d.io.write_triangle_mesh(path + '/' + filename, p_mesh_crop)
    
    mesh = pv.read(path + '/' + filename)
    meshfix = mf.MeshFix(mesh)
    meshfix.repair()
    meshfix.mesh.save(path + '/' + filename)
    
    mesh = o3d.io.read_triangle_mesh(path + '/' + filename)
    simple = mesh.filter_smooth_simple(3)
    taubin = simple.filter_smooth_taubin(2) 
    
    o3d.io.write_triangle_mesh(path + '/' + filename, taubin)
    mesh = pv.read(path + '/' + filename)
    
    return mesh

In [12]:
def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path + '/' +'lod_'+ str(i) + extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

In [14]:
def point_cloud_processing(sequence_id, patient, path):
    pcds = import_pointcloud(sequence_id,path)
    new_pc = full_registration(pcds)
    mesh = reconstruction(new_pc, patient, path)
    panel_draw(mesh)